# 데이터 수집

## 라이브러리 임포트 및 환경변수 설정

In [1]:
pip install requests python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json
import os
from dotenv import load_dotenv
from pydantic import BaseModel, ValidationError
from typing import List
from datetime import datetime

load_dotenv()
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")

## 출력 데이터 형식 정의
### json 데이터 구조 정의 - Pydantic
```
    "category": "카테고리/키워드",
    "articles": [
        {
            "news_url": "기사 URL",
            "title": "기사 제목",
            "text": "기사 본문 (최소 100자 이상의 상세한 내용)",
            "date": "기사 발행일"
        }
    ]
```

In [3]:
class Article(BaseModel):
    news_url: str
    title: str
    text: str
    date: str

class NewsData(BaseModel):
    category: str
    articles: List[Article]

## 프롬프트 생성 함수
1. company_config.json 파일에서 회사별 guideline, sources, aliases 불러오기
2. 각각을 적용시켜 프롬프트 생성

In [4]:
def get_company_info(category, config):
    """카테고리에 맞는 회사 정보 반환 (JSON 파일에서 읽기, 별칭 지원)"""
    category_upper = category.upper()
    companies = config["companies"]
    default_sources = config["default_sources"]
    
    # 1. 정확한 회사명 매치 확인
    if category_upper in companies:
        return companies[category_upper]
    
    # 2. 별칭(aliases)으로 매치 확인
    for company, info in companies.items():
        if "aliases" in info and category_upper in info["aliases"]:
            return info
    
    # 3. 여러 회사가 포함된 경우 처리 (부분 매치)
    matched_info = {"guideline": "", "sources": []}
    for company, info in companies.items():
        if company in category_upper:
            matched_info["guideline"] += f"• {company}: {info['guideline']}\n"
            matched_info["sources"].extend(info["sources"])
    
    if matched_info["guideline"]:
        return matched_info
    
    # 4. 매치되는 회사가 없으면 기본값
    return {
        "guideline": f"• {category}: Focus on latest technical developments, research breakthroughs, product releases, and innovation announcements",
        "sources": default_sources
    }

In [5]:
def get_source_preferences(config):
    """소스 선호도 문자열 반환 (JSON 파일에서 읽기)"""
    source_categories = config.get("source_categories", {})
    
    sources_text = ""
    for i, (category, sources) in enumerate(source_categories.items(), 1):
        # sources가 리스트이므로 문자열로 변환
        sources_str = ", ".join(sources)
        sources_text += f"{i}. {category.replace('_', ' ').title()}: {sources_str}\n"
    
    return sources_text.strip()

In [6]:
def create_tech_news_prompt(category, company_info, source_preferences):
    """기술 뉴스 수집을 위한 프롬프트 생성"""
    
    prompt = f"""
        You are a technology news curator and translator specializing in cutting-edge tech developments.

        Find ALL AVAILABLE latest technology-focused news about "{category}" from recent sources (within 24 hours). Collect as many relevant articles as possible without any numerical limit.

        CONTENT FOCUS:
        • Technical breakthroughs: new AI models, research papers, product launches, API releases
        • Innovation announcements: infrastructure changes, technical partnerships, open-source releases
        • Research developments: academic papers, experimental results, technical demonstrations

        EXCLUSIONS:
        • Business/financial news, stock prices, funding rounds, acquisitions
        • Legal issues, lawsuits, regulatory news, HR/personnel changes
        • Marketing campaigns, user statistics, general company announcements
        • Opinion pieces, analysis without new technical information

        CATEGORY-SPECIFIC GUIDELINES:
        {company_info["guideline"]}

        PREFERRED SOURCES:
        {", ".join(company_info["sources"])}

        SOURCE PREFERENCES:
        {source_preferences}

        URL ACCESSIBILITY REQUIREMENT:
        • ONLY include articles with publicly accessible URLs
        • Exclude paywalled content, subscription-required sites, or restricted access pages
        • Verify that URLs lead to actual article content, not login pages or error pages
        • Prioritize open-access sources and free technical publications

        OUTPUT FORMAT:
        Return ONLY valid JSON with Korean translations:

        {{
            "category": "{category}",
            "articles": [
                {{
                    "news_url": "direct article URL (must be publicly accessible)",
                    "title": "Korean translated title (keep technical terms in English when appropriate)",
                    "text": "Korean summary 200+ characters focusing on technical significance and implications",
                    "date": "actual publication date (YYYY-MM-DD)"
                }}
            ]
        }}

        QUALITY STANDARDS:
        • Collect maximum number of relevant articles available
        • Verify technical accuracy and avoid sensationalized content
        • Prioritize official sources over secondary reporting
        • Double-check URL accessibility before inclusion
        • If no recent technical news found, return empty articles array

        CRITICAL: Output must be valid JSON only. No explanations, markdown, or additional text.
    """
    return prompt

## 데이터 수집 함수

In [7]:
def collect_news_by_date_and_category(category, api_key):
    """
    지정된 날짜와 카테고리에 대한 뉴스 기사를 수집하는 함수
    
    Args:
        category (str): 뉴스 카테고리 (예: "GOOGLE", "META", "OPENAI" 등)
        api_key (str): Perplexity API 키
    
    Returns:
        dict: 카테고리와 기사 목록이 포함된 JSON 형태의 딕셔너리
    """

    if not api_key:
        raise ValueError("환경변수 PERPLEXITY_API_KEY가 설정되지 않았습니다.")
    
    # API 엔드포인트와 헤더 설정
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    # 프롬프트 구성
    with open('./company_config.json', 'r', encoding='utf-8') as f:
        config = json.load(f)

    company_info = get_company_info(category, config)
    source_preferences = get_source_preferences(config)
    prompt = create_tech_news_prompt(category, company_info, source_preferences)

    # 요청 페이로드 정의
    payload = {
        "model": "sonar",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "search_domain_filter": company_info["sources"],
        # 날짜 필터링 추가
        # "search_recency_filter": "day",  # 하루 내의 기사만
        "search_recency_filter": "week",  # 일주일 내의 기사만
        "return_citations": True,  # citations 반환 요청
        "max_tokens": 4000
    }
    
    try:
        # API 호출
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # HTTP 오류 확인
        
        # 응답에서 컨텐츠 추출
        response_data = response.json()
        content = response_data["choices"][0]['message']['content']
        
        # JSON 파싱 시도
        try:
            raw_data = json.loads(content)
            news_data = NewsData(**raw_data)  # pydantic 검증
            return news_data.model_dump()
        except json.JSONDecodeError:
            return {"error": "JSON 파싱 실패", "raw_content": content}
        except ValidationError as ve:
            return {"error": "스키마 검증 실패", "details": json.loads(ve.json())}
            
    except requests.exceptions.RequestException as e:
        return {"error": f"API 요청 실패: {str(e)}"}
    except KeyError as e:
        return {"error": f"응답 형식 오류: {str(e)}"}

## 코드 실행, 데이터 수집 및 출력

In [8]:
if __name__ == "__main__":

    API_KEY = PERPLEXITY_API_KEY
    # 뉴스 수집 실행
    category = "GOOGLE"  # 원하는 카테고리로 변경
    date = datetime.now().strftime("%Y-%m-%d")
    
    result = collect_news_by_date_and_category(category, API_KEY)
    
    # 결과 출력 (보기 좋게 포맷팅)
    print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "category": "GOOGLE",
  "articles": [
    {
      "news_url": "https://research.google/blog/vaultgemma-the-worlds-most-capable-differentially-private-llm/",
      "title": "VaultGemma: 차등 개인 정보 보호가 적용된 세계에서 가장 강력한 LLM",
      "text": "Google Research는 차등 개인 정보 보호(DP)를 적용해 처음부터 학습된 가장 큰 10억 파라미터 규모의 LLM인 VaultGemma를 공개했습니다. 이 모델은 DP의 트레이드오프를 연구한 새로운 확장 법칙에 기반하며, 개인 정보 보호를 핵심으로 하는 AI 개발에 중요한 진전을 의미합니다. 연구 결과와 모델 가중치는 Hugging Face 및 Kaggle에서 공개되어 차세대 개인 정보 보호 AI 연구에 기여합니다.",
      "date": "2025-09-12"
    },
    {
      "news_url": "https://research.google/blog/accelerating-scientific-discovery-with-ai-powered-empirical-software/",
      "title": "AI 기반 실험 소프트웨어로 과학적 발견 가속화",
      "text": "Google Research는 Gemini 모델을 활용해 과학자들이 복잡한 실험 소프트웨어를 자동으로 생성하고 최적화하도록 돕는 AI 시스템을 발표했습니다. 이 시스템은 다양한 분야의 6가지 벤치마크에서 전문가 수준의 성능을 달성하며, 새로운 방법론 및 아키텍처를 구현하고 수천 가지 코드 변형을 탐색해 과학적 가설 평가와 발견 속도를 획기적으로 향상시킵니다.",
      "date": "2025-09-09"
    },
    {
      "news_url": "https://developers.googleblog.com/en